In this notebook, you can consume and see the output of the messages of the kafka topics produced in the scripts/run-flink-sentiment.sh. Execute the `./scripts/run-flink-sentiment.sh` script at the project root after creating table below.

In [ ]:
# jupyterlab_vim

In [ ]:
from kafka import KafkaConsumer
from kafka.admin import KafkaAdminClient
from kafka import KafkaConsumer
from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    Field,
    FloatType,
    DoubleType,
    LongType,
    StringType,
    NestedField,
    StructType,
)

In [ ]:
catalog = load_catalog('iceberg', **{
    'uri': 'http://iceberg-rest:8181',
    'warehouse': 's3a://iceberg/wh',
    's3.endpoint': 'http://minio:9000',
    's3.path-style-access': 'true',
    "py-io-impl": "pyiceberg.io.pyarrow.PyArrowFileIO",
    "s3.access-key-id": "admin",
    "s3.secret-access-key": "password",
    })

In [ ]:
ns = 'flink'
table_name = f"{ns}.sentiment"
catalog.create_namespace_if_not_exists(ns)
schema = Schema(
    NestedField(1, "message", StringType()),
    NestedField(2, "sentiment", StringType()),
    NestedField(3, "timestamp", LongType()),
    NestedField(4, "processed_at", StringType()),
)
#catalog.drop_table('flink.sentiment')
catalog.create_table(
    identifier=table_name,
    schema=schema,
    location="s3://iceberg/wh/",
)

In [ ]:
!!!!Now,Execute the `./scripts/run-flink-sentiment.sh` script at the project root.

## input into kafka source topic


In [ ]:
# verify messages in topic
bootstrap_servers = "kafka-broker:9092"

admin_client = KafkaAdminClient(
    bootstrap_servers=bootstrap_servers,
    client_id="demo",
)
topic_name = "message"
source_topic = KafkaConsumer(
    topic_name,
    bootstrap_servers=["kafka-broker:9092"],
    auto_offset_reset="earliest",
    enable_auto_commit=False,
)

batch = source_topic.poll(timeout_ms=1000)
for partition, messages in batch.items():
    for message in messages:
        print(message.value.decode("utf-8"))
source_topic.close()

In [ ]:
## output of sentiment analysis

In [ ]:
result_topic = KafkaConsumer(
    "sentiment_analysis",
    bootstrap_servers=["kafka-broker:9092"],
    auto_offset_reset="earliest",
    enable_auto_commit=False,
)

result_batch = result_topic.poll(timeout_ms=1000)
for partition, messages in result_batch.items():
    for message in messages:
        print(message.value.decode("utf-8"))
source_topic.close()

In [ ]:

if not catalog.table_exists("flink.sentiment"):
    raise
table = catalog.load_table(table_name)

In [ ]:
df = table.scan(
    limit=10,
).to_pandas()
df